In [36]:
msg_file=r'E:/CrtFile/RES_MSG_FILE_20171115'
f = open(msg_file)
all_raw = f.read()
conv_list = [ conv for conv in  all_raw.split('#'*77+'\n')]
len(conv_list)

294

In [37]:
conv_list[10]

'800084933 1495806852 40039835 1 5 2017-11-15 09:13:32 2017-11-15 02:14:24\n[2017.11.15 02:14:24] 40039835 -> 800084933 [ \xe6\x82\xa8\xe5\xa5\xbd\xef\xbc\x8c\xe4\xbc\x81\xe4\xb8\x9a\xe5\xae\xa2\xe6\x9c\x8dQQ\xe6\x8f\x90\xe7\xa4\xba\xe8\xbf\x98\xe6\x9c\x8924\xe5\xa4\xa9\xe8\xbf\x87\xe6\x9c\x9f\xe6\x98\xaf\xe4\xbb\x80\xe4\xb9\x88\xe6\x84\x8f\xe6\x80\x9d\xef\xbc\x9f ]\n[2017.11.15 02:14:33] 40039835 -> 800084933 [ \xe6\x82\xa8\xe5\xa5\xbd\xef\xbc\x8c\xe4\xbc\x81\xe4\xb8\x9a\xe5\xae\xa2\xe6\x9c\x8dQQ\xe6\x8f\x90\xe7\xa4\xba\xe8\xbf\x98\xe6\x9c\x8924\xe5\xa4\xa9\xe8\xbf\x87\xe6\x9c\x9f\xe6\x98\xaf\xe4\xbb\x80\xe4\xb9\x88\xe6\x84\x8f\xe6\x80\x9d\xef\xbc\x9f ]\n[2017.11.15 02:14:35] 40039835 -> 800084933 [ \xe6\x82\xa8\xe5\xa5\xbd\xef\xbc\x8c\xe4\xbc\x81\xe4\xb8\x9a\xe5\xae\xa2\xe6\x9c\x8dQQ\xe6\x8f\x90\xe7\xa4\xba\xe8\xbf\x98\xe6\x9c\x8924\xe5\xa4\xa9\xe8\xbf\x87\xe6\x9c\x9f\xe6\x98\xaf\xe4\xbb\x80\xe4\xb9\x88\xe6\x84\x8f\xe6\x80\x9d\xef\xbc\x9f ]\n[2017.11.15 02:14:37] 40039835 -> 80008493

In [110]:
def IsAutoReply(msg):
    res = re.search(r'您好，.*为您服务。', msg)
    if res != None:
        return True
    
    res = re.search(r'转接成功.*为您服务。', msg)
    if res != None:
        return True
    
    res = re.search(r'正在为您转接到客服.*请稍候…', msg)
    if res != None:
        return True
    #print msg, res!=None
    return res != None

IsAutoReply('正在为您转接到客服韩鹏-售前咨询，请稍候…')

True

In [118]:
import datetime

def date_diff(t1, t2):
    if len(t1) < 10:
        t1 = '2020.01.01 00:00:00'
    if len(t2) < 10:
        t2 = '2020.01.01 00:00:00'
        
    d1 = datetime.datetime.strptime(t1, "%Y.%m.%d %H:%M:%S")
    d2 = datetime.datetime.strptime(t2, "%Y.%m.%d %H:%M:%S")
    diff_sec = (d1-d2).total_seconds()
    return diff_sec
    
t1 = '2017.11.15 02:14:24'
t2 = '2017.11.15 09:13:31'

date_diff(t1, t2)

-25147.0

In [132]:
import re 

def parse_one_qq(conv):
    qq_msg = []
    msg_c2b = 0
    msg_b2c = 0
    line_num = 0
    first_send_date = ''
    first_recv_date = ''
    
    for line in conv.split('\n'):
        if len(line.strip()) ==0 :
            continue
        
        line_num +=1
        if line_num == 1:
            #kfuin, extuin, qq, send_msg_num, recv_msg_num, first_send_date, first_recv_date = line.split(' ')
            a = line.split(' ')
            #print a 
            kfuin = a[0]
            extuin = a[1]
            qq = a[2]
            send_msg_num = a[3]
            recv_msg_num = a[4]
            #print kfuin, recv_msg_num
        else:
            pat=r'^\[(?P<msg_time>[0-9.: ]*)\] (?P<from_uin>[0-9]+) -> (?P<to_uin>[0-9]+) \[(?P<msg>.*)\]$'
        #    print line 
            #res = re.match(pat, line)
            res = re.search(pat, line)
            if res is not None: 
                msg_time = res.groupdict()['msg_time']
                from_uin = res.groupdict()['from_uin']
                to_uin = res.groupdict()['to_uin']
                msg = res.groupdict()['msg']
                
                if from_uin == qq:
                    if len(first_recv_date) == 0:
                        first_recv_date = msg_time
                    #print msg_time,from_uin,to_uin,msg
                    msg_c2b += 1
                    if msg not in qq_msg:
                        qq_msg.append(msg)
                else:
                    if not IsAutoReply(msg):
                        msg_b2c += 1
                        if len(first_send_date) == 0:
                            first_send_date = msg_time
                            
            
        
        
    if line_num == 0 or msg_c2b == 0:
        return 
    
    dict_conv = {}
    dict_conv['kfuin'] = kfuin
    dict_conv['qq'] = qq
    dict_conv['msg_c2b'] = msg_c2b
    dict_conv['msg_b2c'] = msg_b2c
    dict_conv['recv_msg_num'] = recv_msg_num 
    dict_conv['send_msg_num'] = send_msg_num
    dict_conv['first_recv_date'] = first_recv_date 
    dict_conv['first_send_date'] = first_send_date
    dict_conv['msg'] = ' '.join(qq_msg) 
    
    # 发现和上报数据的统计结果不同时，打印
    #if msg_c2b!=int(recv_msg_num) or msg_b2c != int(send_msg_num):
    #    print kfuin,qq,msg_c2b,msg_b2c,recv_msg_num,send_msg_num,first_recv_date,first_send_date
    #print 'qq_msg', qq_msg
    
    return dict_conv
    
    # 如果超时30分钟的话，打印
    diff_sec = date_diff(first_send_date, first_recv_date) 
    if diff_sec > 0 and diff_sec > 1800:
        #print kfuin,qq,msg_c2b,msg_b2c,recv_msg_num,send_msg_num,first_recv_date,first_send_date
        return dict_conv
    
parse_one_qq(conv_list[5])

{'first_recv_date': '2017.11.15 00:42:15',
 'first_send_date': '2017.11.15 09:13:06',
 'kfuin': '800084933',
 'msg': ' \xe4\xbd\xa0\xe5\xa5\xbd\xe3\x80\x82   \xe6\x89\xbe\xe5\xae\xa2\xe6\x9c\x8d\xe6\xb2\xa1\xe6\x89\xbe\xe5\x88\xb0\xef\xbc\x8c\xe9\x97\xae\xe9\xa2\x98\xe6\x98\xaf\xe3\x80\x82 \xe6\x88\x91\xe7\x9a\x84\xe4\xbc\x81\xe4\xb8\x9aqq\xe5\x88\x9b\xe7\x9a\x84\xe7\xbe\xa4\xe3\x80\x82qq\xe7\x94\xa8\xe6\x88\xb7\xe6\x80\x8e\xe4\xb9\x88\xe6\x90\x9c\xe7\xb4\xa2\xe4\xb8\x8d\xe5\x88\xb0\xef\xbc\x8c\xe3\x80\x82   \xe6\xb2\xa1\xe6\x9c\x89\xe6\x8e\x92\xe5\x90\x8d\xe3\x80\x82 ',
 'msg_b2c': 1,
 'msg_c2b': 4,
 'qq': '1935845748',
 'recv_msg_num': '4',
 'send_msg_num': '1'}

In [133]:
for conv in conv_list:
    d = parse_one_qq(conv)
    print d

In [141]:
import pandas as pd 
df_list = []
for conv in conv_list:
    d = parse_one_qq(conv)
    if d is not None:
        df_list.append(d)

df = pd.DataFrame(df_list, columns=['kfuin','qq','msg_c2b','msg_b2c','recv_msg_num','send_msg_num','first_recv_date','first_send_date','msg'])   
#df.columns=['kfuin','qq','msg_c2b','msg_b2c','recv_msg_num','send_msg_num','first_recv_date','first_send_date','msg']
df

In [145]:
df.head(20)

,kfuin,qq,msg_c2b,msg_b2c,recv_msg_num,send_msg_num,first_recv_date,first_send_date,msg
0,800084933,1935845748,4,1,4,1,2017.11.15 00:42:15,2017.11.15 09:13:06,你好。 找客服没找到，问题是。 我的企业qq创的群。qq用户怎么搜索不到，。 没有...
1,800084933,1402937854,2,1,2,1,2017.11.15 00:43:26,2017.11.15 09:13:08,在么？ QQ被永久封号能不能解封？
2,800084933,1475491951,18,1,18,1,2017.11.15 01:10:38,2017.11.15 09:13:13,你好 我有两个微信号，一个QQ注册的一个手机号注册的然后我不小心把QQ注册的那个微信号...
3,800084933,2903346005,2,1,2,1,2017.11.15 01:25:19,2017.11.15 09:13:26,可以买q号吗
4,800084933,40039835,4,1,5,1,2017.11.15 02:14:24,2017.11.15 09:13:31,您好，企业客服QQ提示还有24天过期是什么意思？
5,800084933,1584920619,3,1,3,11,2017.11.15 04:16:35,2017.11.15 09:13:53,在 搞毛线搞这么多人排队 企业QQ手机可以用吗
6,800084933,1822652929,8,1,8,1,2017.11.15 04:18:58,2017.11.15 09:13:56,在吗 我DNF制裁是怎么会是 我挤线给我制裁 草 你吗的 能不能玩 ...
7,800084933,1410451226,44,1,44,1,2017.11.15 04:59:03,2017.11.15 09:14:00,QQ被盗了 手机号码也被改了 QQ被盗 怎么找回 .. QQ被盗了 ...
8,800084933,2881843009,1,1,1,4,2017.11.15 08:07:30,2017.11.15 09:14:14,后台网站
9,800084933,3445737175,5,5,15,13,2017.11.15 08:48:43,2017.11.15 08:59:57,在吗 我购买是的账号可以解绑换别的吗 时 企业QQ 不是买错了 是把界面...


In [154]:
import jieba
words = [ w for w in jieba.cut(msg) for msg in df['msg'] ]
#for w in words:
    #print w
print len(words)

672


In [23]:
M = [ [1,2,3], [4,5,6], [7,8,9]]
M

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [25]:
M[2][1]

8

In [29]:
[ row[1]+10 for row in M if row[1]%2==1 ]

[15]

In [41]:
L = [1,2,3]
# M=L

import sys 
sys.getrefcount(L)

2

In [43]:
'abc'  'ABC'

'abcABC'

In [48]:
print('a\n\rb\tc')

a
b	c
